In [1]:
import os
import numpy as np
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID" 
os.environ["CUDA_VISIBLE_DEVICES"] = "1" 

In [41]:
file1 = open('../../datasets/Reuters-21578/predictLabels/label_user.txt', 'r')
documents = file1.readlines()  

In [42]:
predict_label_space = []
for doc in documents:
    predict_label_space.append(doc.strip())

In [44]:
file2 = open('../../datasets/Reuters-21578/test_label.txt', 'r')
labels = file2.readlines()  

In [56]:
true_label = []
for row in labels:
    label_list = row.strip().split(' ')
    for label in label_list:
        if not label in true_label:
            true_label.append(label)
true_label

['trade',
 'grain',
 'nat-gas',
 'crude',
 'rubber',
 'tin',
 'sugar',
 'corn',
 'rice',
 'palm-oil',
 'veg-oil',
 'ship',
 'coffee',
 'lumber',
 'wheat',
 'gold',
 'acq',
 'interest',
 'money-fx',
 'copper',
 'ipi',
 'soybean',
 'oilseed',
 'carcass',
 'livestock',
 'earn',
 'bop',
 'lead',
 'gas',
 'jobs',
 'zinc',
 'cpi',
 'gnp',
 'soy-oil',
 'yen',
 'dlr',
 'nickel',
 'groundnut',
 'heat',
 'sunseed',
 'sorghum',
 'pet-chem',
 'cocoa',
 'rapeseed',
 'cotton',
 'money-supply',
 'iron-steel',
 'l-cattle',
 'alum',
 'palladium',
 'strategic-metal',
 'platinum',
 'reserves',
 'meal-feed',
 'sun-oil',
 'rape-oil',
 'groundnut-oil',
 'lin-oil',
 'sun-meal',
 'hog',
 'barley',
 'potato',
 'orange',
 'retail',
 'soy-meal',
 'oat',
 'cotton-oil',
 'fuel',
 'silver',
 'income',
 'wpi',
 'tea',
 'lei',
 'coconut-oil',
 'copra-cake',
 'coconut',
 'dmk',
 'dfl',
 'naphtha',
 'propane',
 'instal-debt',
 'nzdlr',
 'housing',
 'nkr',
 'rye',
 'castor-oil',
 'jet',
 'palmkernel',
 'cpu',
 'rand']

In [6]:
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

query_embedding = model.encode('Computer Science')
passage_embedding = model.encode(['Computational Science',
                                  'Computer Science'])

print("Similarity:", util.dot_score(query_embedding, passage_embedding))

Similarity: tensor([[0.7585, 1.0000]])


In [47]:
passage_embedding = model.encode(true_label)

In [48]:
sim_matrix = np.empty((0,len(true_label)))
for i in range(len(predict_label_space)):
    query_embedding = model.encode(predict_label_space[i])
    sim_matrix = np.append(sim_matrix, util.dot_score(query_embedding, passage_embedding).numpy(), 0)

In [49]:
sim_matrix.shape

(85, 90)

In [50]:
sim_array = sim_matrix.max(axis=1)

In [54]:
count = 0
for sim in sim_array:
    if sim > 0.6:
        count +=1
count

15

In [55]:
sim_list = []
for i, sim_score in enumerate(sim_matrix):
    for j in range(len(sim_score)):
        if sim_score[j] > 0.6:
            sim = [i, predict_label_space[i], j , true_label[j], sim_score[j]]
            sim_list.append(sim)
sim_list

[[11, 'Retail Sales', 63, 'retail', 0.8805943727493286],
 [13, 'Earnings', 25, 'earn', 0.6168959140777588],
 [13, 'Earnings', 69, 'income', 0.6541888117790222],
 [15, 'Oil and Gas', 28, 'gas', 0.7158282399177551],
 [15, 'Oil and Gas', 67, 'fuel', 0.6818692088127136],
 [19, 'Shipping', 11, 'ship', 0.7106267213821411],
 [25, 'Agriculture', 24, 'livestock', 0.6434237957000732],
 [30, 'Coffee', 12, 'coffee', 0.9999999403953552],
 [30, 'Coffee', 71, 'tea', 0.6160335540771484],
 [37, 'Real Estate', 82, 'housing', 0.6607487201690674],
 [40, 'Soybean Exports', 21, 'soybean', 0.690209686756134],
 [44, 'Metal Trading', 50, 'strategic-metal', 0.7207976579666138],
 [46, 'Debt', 80, 'instal-debt', 0.6317770481109619],
 [53, 'Coffee Prices', 12, 'coffee', 0.6282552480697632],
 [65, 'Gas Supplies', 28, 'gas', 0.6856974363327026],
 [73, 'Wheat Harvest', 14, 'wheat', 0.7438153624534607],
 [74, 'Shipbuilding', 11, 'ship', 0.6174918413162231],
 [83, 'Trade', 0, 'trade', 0.9999999403953552]]